In [1]:
### 2021 Census Data (ACS 5-year)for Selected Variables - State of MD Totals



In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
import censusgeocode as cg
import cenpy
import gmaps
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Census & gmaps API Keys
# the latest year available is the default year and so you do not need to specify the year

from config import (api_key, gkey)
c = Census(api_key)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [3]:
import os

In [4]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Commercial_corridors\\LUC_Corr_Analysis'

In [5]:
pd.set_option('display.max_rows', 200)

In [6]:
pd.set_option('max_colwidth', 100)

In [7]:
# Save the acreage/ square mile data (csv file) in variable

data_SQMI = "MD_SQ_mile.csv"

In [8]:
# Create dataframe by reading the variable data from the code block above
MD_SQMI_df = pd.read_csv(data_SQMI)
MD_SQMI_df

,Name,Area_Acres,Area_mi2
0,Maryland,7940500,12407


In [9]:
# American Community Survey 5-Year Data DETAILED Tables 
# The data are population COUNTS

# See "When to Use 1-year or 5-year estimates": 
# https://www.census.gov/programs-surveys/acs/guidance/estimates.html

# Set this to false if you're trying to do this without an internet connection
# and data which would have been fetched from an API query will be read from cached files instead
INTERNET_IS_WORKING = True

if INTERNET_IS_WORKING:
    con = cenpy.remote.APIConnection('ACSDT5Y2021')
    variables = con.variables
else:
    variables = pd.read_csv('data/ACSDT5Y2021_variables.csv',index_col='Unnamed: 0')

# No matter which product you use, a cenpy APIConnection can show you the variables 
# which it can retrieve for you, returned as a pandas DataFrame.
print(f"ACSDT5Y2021 provides {len(variables)} variables.") # how many are there?
variables.head(20)

ACSDT5Y2021 provides 27927 variables.


,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN
B24022_060E,Estimate!!Total:!!Female:!!Service occupations:!!Food preparation and serving related occupations,SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS)...,int,B24022,0,NaN,NaN,"B24022_060EA,B24022_060M,B24022_060MA",NaN
B19001B_014E,"Estimate!!Total:!!$100,000 to $124,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AM...,int,B19001B,0,NaN,NaN,"B19001B_014EA,B19001B_014M,B19001B_014MA",NaN
B07007PR_019E,Estimate!!Total:!!Moved from different municipio:!!Foreign born:!!Naturalized U.S. citizen,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY CITIZENSHIP STATUS FOR CURRENT RESIDENCE IN PUERTO RICO,int,B07007PR,0,NaN,NaN,"B07007PR_019EA,B07007PR_019M,B07007PR_019MA",NaN
B19101A_004E,"Estimate!!Total:!!$15,000 to $19,999",FAMILY INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS) (WHITE ALONE HOUSEHOLDER),int,B19101A,0,NaN,NaN,"B19101A_004EA,B19101A_004M,B19101A_004MA",NaN
B24022_061E,Estimate!!Total:!!Female:!!Service occupations:!!Building and grounds cleaning and maintenance o...,SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS)...,int,B24022,0,NaN,NaN,"B24022_061EA,B24022_061M,B24022_061MA",NaN
B19001B_013E,"Estimate!!Total:!!$75,000 to $99,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AM...,int,B19001B,0,NaN,NaN,"B19001B_013EA,B19001B_013M,B19001B_013MA",NaN
B07007PR_018E,Estimate!!Total:!!Moved from different municipio:!!Foreign born:,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY CITIZENSHIP STATUS FOR CURRENT RESIDENCE IN PUERTO RICO,int,B07007PR,0,NaN,NaN,"B07007PR_018EA,B07007PR_018M,B07007PR_018MA",NaN


In [10]:
# Run Census Search to retrieve data on Baltimore City, MD 
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
# The state FIPS code for MD is 24 and the FIPS code for Balt City is 510
census_data = c.acs5.state(("NAME", "B19001_001E","B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",
                          "B15003_016E",               
                          "B15003_017E",
                          "B15003_021E",               
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_005E",              
                          "B02001_008E",
                          "B03001_003E",
                          "B05002_013E",
                          "B25008_002E",
                          "B25003_002E",                                    
                          "B25003_003E",
                          "B28003_005E",
                          "B28003_006E",
                          "C24030_006E",                
                          "C24030_007E", 
                          "C24030_009E",                
                          "C24030_010E",                
                          "C24030_014E",                
                          "C24030_017E",                
                          "C24030_021E",                
                          "C24030_024E",
                          "C24030_028E",              
                          "C24030_033E",                
                          "C24030_034E",                
                          "C24030_036E",                
                          "C24030_037E",                
                          "C24030_041E",  
                          "C24030_044E",               
                          "C24030_048E",                
                          "C24030_051E",
                          "C24030_055E"),          
                          state_fips = "24"
                          )
census_pd = pd.DataFrame(census_data)
census_pd.head()      

,NAME,B19001_001E,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_016E,...,C24030_033E,C24030_034E,C24030_036E,C24030_037E,C24030_041E,C24030_044E,C24030_048E,C24030_051E,C24030_055E,state
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,21744.0,40345.0,140151.0,34702.0,98332.0,199170.0,534106.0,131037.0,161448.0,24


In [11]:
# Pull values from the ACS 5 yr. census codes/ variables listed and save them in the variable "age_data"

age_data = c.acs5.state(("NAME", "B01001_003E",                                    
                          "B01001_004E",
                          "B01001_005E",
                          "B01001_006E",
                          "B01001_007E",
                          "B01001_008E",
                          "B01001_009E",
                          "B01001_010E",
                          "B01001_011E",
                          "B01001_012E",
                          "B01001_013E",
                          "B01001_014E",
                          "B01001_015E",
                          "B01001_016E",            
                          "B01001_017E",
                          "B01001_018E",
                          "B01001_019E",
                          "B01001_020E",
                          "B01001_021E",
                          "B01001_022E",
                          "B01001_023E",
                          "B01001_024E",
                          "B01001_025E",
                          
                          "B01001_027E",
                          "B01001_028E",
                          "B01001_029E",
                          "B01001_030E",
                          "B01001_031E",
                          "B01001_032E",
                          "B01001_033E",
                          "B01001_034E",
                          "B01001_035E",
                          "B01001_036E",
                          "B01001_037E",
                          "B01001_038E",
                          "B01001_039E",            
                          "B01001_040E",
                          "B01001_041E",
                          "B01001_042E",
                          "B01001_043E",
                          "B01001_044E",
                          "B01001_045E",
                          "B01001_046E",
                          "B01001_047E",
                          "B01001_048E",
                          "B01001_049E"),          
                                     
                          state_fips = "24",
                          )
age_pd = pd.DataFrame(age_data)
age_pd.head()      

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_041E,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state
0,Maryland,186840.0,190426.0,191702.0,116895.0,79087.0,41166.0,39688.0,117449.0,209055.0,...,220599.0,81418.0,115354.0,69024.0,91215.0,118940.0,86661.0,60222.0,71409.0,24


In [12]:
# Sum columns of age groups that are under 18 years old for male and female and add new column "Pop. <18 years"
columns_under18 = ["B01001_003E",                                    
                   "B01001_004E",
                   "B01001_005E",
                   "B01001_006E",
                   "B01001_027E",
                   "B01001_028E",
                   "B01001_029E",
                   "B01001_030E"]
age_pd['Pop. <18 years']= age_pd[columns_under18].sum(axis=1)
age_pd.head()

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,Pop. <18 years
0,Maryland,186840.0,190426.0,191702.0,116895.0,79087.0,41166.0,39688.0,117449.0,209055.0,...,81418.0,115354.0,69024.0,91215.0,118940.0,86661.0,60222.0,71409.0,24,1344177.0


In [13]:
# Sum columns of age groups that are 18-64 years old (working age) for male and female and add new column "Pop. working age"
columns_working_age = [                                    
                   "B01001_007E",
                   "B01001_008E",
                   "B01001_009E",
                   "B01001_010E",
                   "B01001_011E",
                   "B01001_012E",   
                   "B01001_013E",
                   "B01001_014E",    
                   "B01001_015E",    
                   "B01001_016E",   
                   "B01001_017E", 
                   "B01001_018E",    
                   "B01001_019E",    
                   "B01001_031E",
                   "B01001_032E",
                   "B01001_033E",
                   "B01001_034E",
                   "B01001_035E",   
                   "B01001_036E",
                   "B01001_037E",    
                   "B01001_038E",    
                   "B01001_039E",   
                   "B01001_040E", 
                   "B01001_041E",    
                   "B01001_042E",    
                   "B01001_043E"]    
                           
age_pd['Pop. working age']= age_pd[columns_working_age].sum(axis=1)
age_pd.head()         
            
               


,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,Pop. <18 years,Pop. working age
0,Maryland,186840.0,190426.0,191702.0,116895.0,79087.0,41166.0,39688.0,117449.0,209055.0,...,115354.0,69024.0,91215.0,118940.0,86661.0,60222.0,71409.0,24,1344177.0,3783921.0


In [14]:
# Sum columns of age groups that are 65+ years old for male and female and add new column "Pop. 65+ years"
columns_senior = ["B01001_020E",
                  "B01001_021E",
                  "B01001_022E",
                  "B01001_023E",                                    
                  "B01001_024E",
                  "B01001_025E",
                  "B01001_044E",
                  "B01001_045E",
                  "B01001_046E",
                  "B01001_047E",                                    
                  "B01001_048E",
                  "B01001_049E"]               
                          
age_pd['Pop. 65+ years']= age_pd[columns_senior].sum(axis=1)
age_pd.head()                           

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Maryland,186840.0,190426.0,191702.0,116895.0,79087.0,41166.0,39688.0,117449.0,209055.0,...,69024.0,91215.0,118940.0,86661.0,60222.0,71409.0,24,1344177.0,3783921.0,875337.0


In [15]:
age_temp = age_pd[[ "NAME","state", "Pop. <18 years", "Pop. working age", "Pop. 65+ years"]]
age_temp.head()

,NAME,state,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Maryland,24,1344177.0,3783921.0,875337.0


In [16]:
# Rename column so can merge on it later
age_temp = age_temp.rename(columns={"NAME": "Name"
                                   })
age_temp

,Name,state,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Maryland,24,1344177.0,3783921.0,875337.0


In [17]:
age_temp.dtypes

Name                 object
state                object
Pop. <18 years      float64
Pop. working age    float64
Pop. 65+ years      float64
dtype: object

In [18]:
age_final = age_temp.copy()

In [19]:
# Did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          ) 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state
0,Maryland,2757200.0,24


In [20]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's understandable
census_pd = census_pd.rename(columns={"B19001_001E": "Total households",
                                      "B01003_001E": "Population",                                        
                                      "B01002_001E": "Median age",
                                      "B19013_001E": "Median household income",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_016E": "# persons 12th grade, no diploma",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_021E": "# persons Associate's degree",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_005E": "Pop. Asian",        
                                      "B02001_008E": "Pop. two or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B05002_013E": "# Foreign-born",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "B28003_005E": "# households with computer, no internet subscription",
                                      "B28003_006E": "# households no computer",
                                      "C24030_006E": "Construction-M", 
                                      "C24030_007E": "Manufacturing-M", 
                                      "C24030_009E": "Retail Trade-M",
                                      "C24030_010E": "Transportation, warehousing, utilities-M",
                                      "C24030_014E": "Finance, insurance, real estate-M",               
                                      "C24030_017E": "Professional, scientific, mgmt, administrative-M",              
                                      "C24030_021E": "Educational services, health care, social assistance-M",                
                                      "C24030_024E": "Accommodations, food services, arts, entertainment-M",
                                      "C24030_028E": "Public administration-M",
                                      "C24030_033E": "Construction",                
                                      "C24030_034E": "Manufacturing",
                                      "C24030_036E": "Retail Trade",                
                                      "C24030_037E": "Transportation, warehousing, utilities",                
                                      "C24030_041E": "Finance, insurance, real estate",  
                                      "C24030_044E": "Professional, scientific, mgmt, administrative",               
                                      "C24030_048E": "Educational services, health care, social assistance",
                                      "C24030_051E": "Accommodations, food services, arts, entertainment",
                                      "C24030_055E": "Public administration",
                                      "NAME": "Name",
                                      "state": "State"
                                     })
census_pd

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Construction,Manufacturing,Retail Trade,"Transportation, warehousing, utilities","Finance, insurance, real estate","Professional, scientific, mgmt, administrative","Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,State
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,21744.0,40345.0,140151.0,34702.0,98332.0,199170.0,534106.0,131037.0,161448.0,24


In [21]:
census_pd["Construction"] = census_pd["Construction-M"].astype(
        int) + census_pd["Construction"].astype(
        int) 
census_pd.head()

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Construction,Manufacturing,Retail Trade,"Transportation, warehousing, utilities","Finance, insurance, real estate","Professional, scientific, mgmt, administrative","Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,State
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,207235,40345.0,140151.0,34702.0,98332.0,199170.0,534106.0,131037.0,161448.0,24


In [22]:
census_pd["Manufacturing"] = census_pd["Manufacturing-M"].astype(
        int) + census_pd["Manufacturing"].astype(
        int) 

In [23]:
census_pd["Retail Trade"] = census_pd["Retail Trade-M"].astype(
        int) + census_pd["Retail Trade"].astype(
        int) 

In [24]:
census_pd["Transportation, warehousing, utilities"] = census_pd["Transportation, warehousing, utilities-M"].astype(
        int) + census_pd["Transportation, warehousing, utilities"].astype(
        int) 

In [25]:
census_pd["Finance, insurance, real estate"] = census_pd["Finance, insurance, real estate-M"].astype(
        int) + census_pd["Finance, insurance, real estate"].astype(
        int) 

In [26]:
census_pd["Professional, scientific, mgmt, administrative"] = census_pd["Professional, scientific, mgmt, administrative-M"].astype(
        int) + census_pd["Professional, scientific, mgmt, administrative"].astype(
        int) 

In [27]:
census_pd["Educational services, health care, social assistance"] = census_pd["Educational services, health care, social assistance-M"].astype(
        int) + census_pd["Educational services, health care, social assistance"].astype(
        int) 

In [28]:
census_pd["Accommodations, food services, arts, entertainment"] = census_pd["Accommodations, food services, arts, entertainment-M"].astype(
        int) + census_pd["Accommodations, food services, arts, entertainment"].astype(
        int) 

In [29]:
census_pd["Public administration"] = census_pd["Public administration-M"].astype(
        int) + census_pd["Public administration"].astype(
        int) 
census_pd

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Construction,Manufacturing,Retail Trade,"Transportation, warehousing, utilities","Finance, insurance, real estate","Professional, scientific, mgmt, administrative","Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,State
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,207235,135715,291465,141078,184340,473202,726217,258134,331518,24


In [30]:
# merge the census_pd dataframe with the age_final dataframe on the common column "NAME"
census_MD = pd.merge(
    census_pd, age_final, on="Name"
                        )
census_MD

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,"Finance, insurance, real estate","Professional, scientific, mgmt, administrative","Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,State,state,Pop. <18 years,Pop. working age,Pop. 65+ years
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,184340,473202,726217,258134,331518,24,24,1344177.0,3783921.0,875337.0


In [31]:
# merge the census_MD dataframe with the MD_SQMI_df dataframe on the common column "Name"
census_MD1 = pd.merge(
    census_MD, MD_SQMI_df, on="Name"
                        )
census_MD1

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,"Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,State,state,Pop. <18 years,Pop. working age,Pop. 65+ years,Area_Acres,Area_mi2
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,726217,258134,331518,24,24,1344177.0,3783921.0,875337.0,7940500,12407


In [32]:
census_MD1.columns

Index(['Name', 'Total households', 'Median household income', 'Population',
       'Median age', 'Per capita income', 'Poverty count',
       'Unemployment count', '# employed, age 16+',
       '# persons 12th grade, no diploma',
       '# persons age 25+ graduated high school',
       '# persons Associate's degree',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. two or more races', 'Pop. Hispanic origin',
       '# Foreign-born', 'Total pop. in occupied housing units by tenure',
       'Total owner-occupied units', 'Total renter-occupied units',
       '# households with computer, no internet subscription',
       '# households no computer', 'Construction-M', 'Manufacturing-M',
       'Retail Trade-M', 'Transportation, warehousing, utilities-M',
       'Finance, insurance, real estate-M',
       'Professional, scientific, mgmt, administrative-M',
       'Educational services, health care, social assistance-M',
       'Accommod

In [33]:
# Drop extraneous columns
census_MD = census_MD1.drop(["State", "state"], axis=1)

In [34]:
census_MD_final = census_MD.copy()

In [35]:
census_MD_final["Total occupied units"] = census_MD_final.loc[:, ["Total owner-occupied units", "Total renter-occupied units"]].sum(axis=1)
census_MD_final

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,"Professional, scientific, mgmt, administrative","Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,Pop. <18 years,Pop. working age,Pop. 65+ years,Area_Acres,Area_mi2,Total occupied units
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,473202,726217,258134,331518,1344177.0,3783921.0,875337.0,7940500,12407,2192518.0


In [36]:
MD_percentages = census_MD_final.copy()
MD_percentages

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,"Professional, scientific, mgmt, administrative","Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,Pop. <18 years,Pop. working age,Pop. 65+ years,Area_Acres,Area_mi2,Total occupied units
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,473202,726217,258134,331518,1344177.0,3783921.0,875337.0,7940500,12407,2192518.0


In [37]:
MD_percentages['Pop. density per Sq Mile'] = MD_percentages['Population'].astype(
                                    float) / MD_percentages['Area_mi2'].astype(
                                    float)
MD_percentages   

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,"Educational services, health care, social assistance","Accommodations, food services, arts, entertainment",Public administration,Pop. <18 years,Pop. working age,Pop. 65+ years,Area_Acres,Area_mi2,Total occupied units,Pop. density per Sq Mile
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,726217,258134,331518,1344177.0,3783921.0,875337.0,7940500,12407,2192518.0,483.874829


In [38]:
MD_percentages["Home ownership rate"] = 100 * \
    MD_percentages["Total owner-occupied units"].astype(
        int) / MD_percentages["Total occupied units"].astype(
        int) 

MD_percentages.head()

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,"Accommodations, food services, arts, entertainment",Public administration,Pop. <18 years,Pop. working age,Pop. 65+ years,Area_Acres,Area_mi2,Total occupied units,Pop. density per Sq Mile,Home ownership rate
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,258134,331518,1344177.0,3783921.0,875337.0,7940500,12407,2192518.0,483.874829,66.769851


In [39]:
# Round the home ownership rate to one decimal point; using "float" instead of "int" because want to use decimal points
MD_percentages["Home ownership rate"] = MD_percentages["Home ownership rate"].astype(float).round(1)


In [40]:
# Add a new column for poverty rate (Poverty Count / Population)
MD_percentages["Poverty rate"] = 100 * \
    MD_percentages["Poverty count"].astype(
        int) / MD_percentages["Population"].astype(int)

MD_percentages["Poverty rate"] = MD_percentages["Poverty rate"].astype(float).round(1)


# Add a new column for unemployment rate (Employment Count / Population)
MD_percentages["Unemployment rate"] = 100 * \
    MD_percentages["Unemployment count"].astype(
        int) / MD_percentages["Population"].astype(int)

MD_percentages["Unemployment rate"] = MD_percentages["Unemployment rate"].astype(float).round(1)

MD_percentages.head()

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Pop. <18 years,Pop. working age,Pop. 65+ years,Area_Acres,Area_mi2,Total occupied units,Pop. density per Sq Mile,Home ownership rate,Poverty rate,Unemployment rate
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,1344177.0,3783921.0,875337.0,7940500,12407,2192518.0,483.874829,66.8,9.2,3.0


In [41]:
MD_percentages["Percent Black"] = 100 * \
    MD_percentages["Pop. Black"].astype(
        int) / MD_percentages["Population"].astype(
        int) 

MD_percentages["Percent Black"] = MD_percentages["Percent Black"].astype(float).round(1)

In [42]:
MD_percentages["Percent white"] = 100 * \
    MD_percentages["Pop. white"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent white"] = MD_percentages["Percent white"].astype(float).round(1)


In [43]:
MD_percentages["Percent Asian"] = 100 * \
    MD_percentages["Pop. Asian"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Asian"] = MD_percentages["Percent Asian"].astype(float).round(1)


In [44]:
MD_percentages["Percent Hispanic origin"] = 100 * \
    MD_percentages["Pop. Hispanic origin"].astype(
        int) /MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Hispanic origin"] = MD_percentages["Percent Hispanic origin"].astype(float).round(1)


In [45]:
MD_percentages["Percent two or more races"] = 100 * \
    MD_percentages["Pop. two or more races"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent two or more races"] = MD_percentages["Percent two or more races"].astype(float).round(1)


In [46]:
MD_percentages["Percent 12th grade, no diploma"] = 100 * \
    MD_percentages["# persons 12th grade, no diploma"].astype(
        int) /MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent 12th grade, no diploma"] = MD_percentages["Percent 12th grade, no diploma"].astype(float).round(1)


In [47]:
MD_percentages["Percent graduated high school"] = 100 * \
    MD_percentages["# persons age 25+ graduated high school"].astype(
        int) /MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent graduated high school"] = MD_percentages["Percent graduated high school"].astype(float).round(1)


In [48]:
MD_percentages["Percent earned Bachelor's degree"] = 100 * \
    MD_percentages["# persons age 25+ with Bachelor's degree"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent earned Bachelor's degree"] = MD_percentages["Percent earned Bachelor's degree"].astype(float).round(1)


In [49]:
MD_percentages["Percent computer, no internet subscription"] = 100 * \
    MD_percentages["# households with computer, no internet subscription"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent computer, no internet subscription"] = MD_percentages["Percent computer, no internet subscription"].astype(float).round(1)


In [50]:
MD_percentages["Percent no computer"] = 100 * \
    MD_percentages["# households no computer"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent no computer"] = MD_percentages["Percent no computer"].astype(float).round(1)


In [51]:
MD_percentages["Percent less than 18 yrs old"] = 100 * \
    MD_percentages["Pop. <18 years"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent less than 18 yrs old"] = MD_percentages["Percent less than 18 yrs old"].astype(float).round(1)


In [52]:
# working age is 18-64 yrs old
MD_percentages["Percent working age"] = 100 * \
    MD_percentages["Pop. working age"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent working age"] = MD_percentages["Percent working age"].astype(float).round(1)


In [53]:
MD_percentages["Percent 65 or older"] = 100 * \
    MD_percentages["Pop. 65+ years"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent 65 or older"] = MD_percentages["Percent 65 or older"].astype(float).round(1)
MD_percentages

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Percent Hispanic origin,Percent two or more races,"Percent 12th grade, no diploma",Percent graduated high school,Percent earned Bachelor's degree,"Percent computer, no internet subscription",Percent no computer,Percent less than 18 yrs old,Percent working age,Percent 65 or older
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,9.8,3.3,1.1,14.9,14.6,2.5,3.2,22.4,63.0,14.6


In [54]:
MD_percentages["Percent Foreign-born"] = 100 * \
    MD_percentages["# Foreign-born"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Foreign-born"] = MD_percentages["Percent Foreign-born"].astype(float).round(1)
MD_percentages

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Percent two or more races,"Percent 12th grade, no diploma",Percent graduated high school,Percent earned Bachelor's degree,"Percent computer, no internet subscription",Percent no computer,Percent less than 18 yrs old,Percent working age,Percent 65 or older,Percent Foreign-born
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,3.3,1.1,14.9,14.6,2.5,3.2,22.4,63.0,14.6,15.1


In [55]:
MD_percentages["Percent Manufacturing"] = 100 * \
    MD_percentages["Manufacturing"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Manufacturing"] = MD_percentages["Percent Manufacturing"].astype(float).round(1)


In [56]:
MD_percentages["Percent Retail Trade"] = 100 * \
    MD_percentages["Retail Trade"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Retail Trade"] = MD_percentages["Percent Retail Trade"].astype(float).round(1)


In [57]:
MD_percentages["Percent Transportation, warehousing, utilities"] = 100 * \
    MD_percentages["Transportation, warehousing, utilities"].astype(
        int) /MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Transportation, warehousing, utilities"] = MD_percentages["Percent Transportation, warehousing, utilities"].astype(float).round(1)

In [58]:
MD_percentages["Percent Finance, insurance, real estate"] = 100 * \
    MD_percentages["Finance, insurance, real estate"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Finance, insurance, real estate"] = MD_percentages["Percent Finance, insurance, real estate"].astype(float).round(1)


In [59]:
MD_percentages["Percent Professional, scientific, mgmt, administrative"] = 100 * \
    MD_percentages["Professional, scientific, mgmt, administrative"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Professional, scientific, mgmt, administrative"] = MD_percentages["Percent Professional, scientific, mgmt, administrative"].astype(float).round(1)


In [60]:
MD_percentages["Percent Educational services, health care, social assistance"] = 100 * \
    MD_percentages["Educational services, health care, social assistance"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Educational services, health care, social assistance"] = MD_percentages["Percent Educational services, health care, social assistance"].astype(float).round(1)

In [61]:
MD_percentages["Percent Accommodations, food services, arts, entertainment"] = 100 * \
    MD_percentages["Accommodations, food services, arts, entertainment"].astype(
        int) / MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Accommodations, food services, arts, entertainment"] = MD_percentages["Percent Accommodations, food services, arts, entertainment"].astype(float).round(1)

In [62]:
MD_percentages["Percent Public administration"] = 100 * \
    MD_percentages["Public administration"].astype(
        int) /MD_percentages["Population"].astype(
        int) 
MD_percentages["Percent Public administration"] = MD_percentages["Percent Public administration"].astype(float).round(1)
MD_percentages

,Name,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Percent 65 or older,Percent Foreign-born,Percent Manufacturing,Percent Retail Trade,"Percent Transportation, warehousing, utilities","Percent Finance, insurance, real estate","Percent Professional, scientific, mgmt, administrative","Percent Educational services, health care, social assistance","Percent Accommodations, food services, arts, entertainment",Percent Public administration
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,14.6,15.1,2.3,4.9,2.3,3.1,7.9,12.1,4.3,5.5


In [63]:
MD_percentages.columns

Index(['Name', 'Total households', 'Median household income', 'Population',
       'Median age', 'Per capita income', 'Poverty count',
       'Unemployment count', '# employed, age 16+',
       '# persons 12th grade, no diploma',
       '# persons age 25+ graduated high school',
       '# persons Associate's degree',
       '# persons age 25+ with Bachelor's degree', 'Pop. white', 'Pop. Black',
       'Pop. Asian', 'Pop. two or more races', 'Pop. Hispanic origin',
       '# Foreign-born', 'Total pop. in occupied housing units by tenure',
       'Total owner-occupied units', 'Total renter-occupied units',
       '# households with computer, no internet subscription',
       '# households no computer', 'Construction-M', 'Manufacturing-M',
       'Retail Trade-M', 'Transportation, warehousing, utilities-M',
       'Finance, insurance, real estate-M',
       'Professional, scientific, mgmt, administrative-M',
       'Educational services, health care, social assistance-M',
       'Accommod

In [64]:
# Replace column name "Name" so we can match it on "Corridor" column at a later step (also rename a few other columns)
MD_percentages = MD_percentages.rename(columns={"Name": "Corridor",
                                               })
MD_percentages

,Corridor,Total households,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+","# persons 12th grade, no diploma",...,Percent 65 or older,Percent Foreign-born,Percent Manufacturing,Percent Retail Trade,"Percent Transportation, warehousing, utilities","Percent Finance, insurance, real estate","Percent Professional, scientific, mgmt, administrative","Percent Educational services, health care, social assistance","Percent Accommodations, food services, arts, entertainment",Percent Public administration
0,Maryland,2192518.0,81868.0,6003435.0,38.6,40517.0,553496.0,181089.0,3051333.0,65932.0,...,14.6,15.1,2.3,4.9,2.3,3.1,7.9,12.1,4.3,5.5


In [65]:
# Export file as an Excel file (will merge this file with the corridor info. in "ACS5YR-detailed-tables-2021.ipynb")
#MD_percentages.to_excel("MD_demographics_2021.xlsx")

In [66]:
# export as csv 
# ignore "percentages" in name - file includes counts as well as percentages
MD_percentages.to_csv("MD_demographics_2021.csv", index = False, header=True)